In [6]:
import pandas as pd

defender_data = pd.read_csv('data/LRD_data.csv')
defender_data.head()

,price_final,model_type,model_variant_final,ext_color_final,mileage_final,age,originality_final
0,49990.0,Land Rover Defender 90,90 Hard Top,Gray,0.000000e+00,33,Restored-Modified
1,91500.0,NAS 5-Speed,90 Soft Top,Green,6.300000e+04,28,Original & Highly Original
2,88995.0,5-Speed,110 Hard Top,Blue,1.560010e+10,31,Restored-Modified
3,57999.0,300TDI 5-Speed,110 Wagon,Blue,1.733625e+05,27,Restored-Original
4,112000.0,5-Speed,90 Soft Top,Black,1.030000e+05,29,Restored-Original


Handling some issues with mileage
--- Problems ---
1) Null, 0 vals --> del
2) Scraped and concactenated km + miles --> 15600096934 --> 156k km == 96k miles so change to 96,934 
3) Added decimal val --> 173362.509 should just be 173362 miles

In [30]:
def kmmiles_to_miles(mileage):
    mileage = str(int(mileage))
    m_len = len(mileage)

    if m_len<6:
        raise ValueError("int too small")
    elif m_len==6:
        new_mileage = int(mileage[3:])
    elif m_len==7 or m_len==8:
        new_mileage= int(mileage[4:])
    elif m_len == 9 or m_len==10:
        new_mileage = int(mileage[5:])
    elif m_len==11 or m_len==12:
        new_mileage = int(mileage[6:])
    else:
        raise ValueError("format not recognized")
    return new_mileage

In [33]:
for i, mileage in enumerate(defender_data['mileage_final']):
    if mileage > 500000:  # Handling concatenated km + miles
        try:
            new_mileage = kmmiles_to_miles(mileage)
        except ValueError:
            print(mileage)
        
        defender_data.at[i, 'mileage_final'] = new_mileage
    elif pd.isna(mileage) or mileage == 0:  
        defender_data.at[i, 'mileage_final'] = None
    else:
        defender_data.at[i, 'mileage_final'] = round(mileage)

defender_data.dropna(subset=['mileage_final'], inplace=True)
defender_data.head()

15600096934.0


NameError: name 'new_mileage' is not defined